# Msoup Closure Model: Baseline Validation

This notebook validates the core Msoup model implementation:

1. **Visibility function** V(M; κ) = exp[-κ(M-2)]
2. **M_vis(z) turn-on** with (ΔM, z_t, w) parameters
3. **Growth equation solver** reducing to ΛCDM when c_*² = 0
4. **Power spectrum suppression** as function of (k, z)
5. **Halo mass function ratio** vs CDM

**Key test**: Does the model reduce exactly to CDM when c_*² = 0?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from msoup_model import (
    MsoupParams, CosmologyParams, MsoupGrowthSolver,
    visibility, m_vis, c_eff_squared, validate_lcdm_limit,
    HaloMassFunction
)

# Set random seed for reproducibility
np.random.seed(42)

# Use a clean style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Visibility Function V(M; κ)

The visibility function quantifies how "visible" or measurable phenomena at order M are.
M = 2 is the threshold where visibility = 1; higher M means exponentially suppressed visibility.

In [ ]:
M_range = np.linspace(0, 6, 100)
kappa_values = [0.5, 1.0, 2.0]

fig, ax = plt.subplots(figsize=(8, 5))
for kappa in kappa_values:
    V = visibility(M_range, kappa=kappa)
    ax.plot(M_range, V, label=f'κ = {kappa}')

ax.axvline(2, color='k', linestyle='--', alpha=0.5, label='M = 2 (threshold)')
ax.set_xlabel('Order statistic M')
ax.set_ylabel('Visibility V(M; κ)')
ax.set_title('Visibility Function: V(M; κ) = exp[-κ(M-2)]')
ax.legend()
ax.set_xlim(0, 6)
ax.set_ylim(0, 1.1)
plt.tight_layout()
plt.savefig('../results/visibility_function.png', dpi=150)
plt.show()

## 2. Visible Order Statistic M_vis(z)

M_vis(z) = 2 + ΔM / (1 + exp[(z - z_t) / w])

This controls when the Msoup effect "turns on":
- At high z >> z_t: M_vis → 2 (full visibility, no effect)
- At low z << z_t: M_vis → 2 + ΔM (above threshold, effect active)

In [ ]:
z_range = np.linspace(0, 10, 200)

# Different parameter choices
param_sets = [
    MsoupParams(Delta_M=0.5, z_t=2.0, w=0.5, c_star_sq=100),
    MsoupParams(Delta_M=1.0, z_t=3.0, w=0.3, c_star_sq=100),
    MsoupParams(Delta_M=0.3, z_t=1.0, w=1.0, c_star_sq=100),
]
labels = [
    r'$\Delta M=0.5, z_t=2, w=0.5$',
    r'$\Delta M=1.0, z_t=3, w=0.3$',
    r'$\Delta M=0.3, z_t=1, w=1.0$',
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# M_vis(z)
for params, label in zip(param_sets, labels):
    M = m_vis(z_range, params)
    ax1.plot(z_range, M, label=label)

ax1.axhline(2, color='k', linestyle='--', alpha=0.5, label='M = 2 (threshold)')
ax1.set_xlabel('Redshift z')
ax1.set_ylabel(r'$M_{vis}(z)$')
ax1.set_title('Visible Order Statistic')
ax1.legend(fontsize=9)
ax1.set_xlim(0, 10)

# c_eff^2(z)
for params, label in zip(param_sets, labels):
    c_eff_sq = c_eff_squared(z_range, params)
    ax2.plot(z_range, c_eff_sq, label=label)

ax2.set_xlabel('Redshift z')
ax2.set_ylabel(r'$c_{eff}^2(z)$ [(km/s)$^2$]')
ax2.set_title('Effective Sound Speed Squared')
ax2.legend(fontsize=9)
ax2.set_xlim(0, 10)

plt.tight_layout()
plt.savefig('../results/m_vis_and_c_eff.png', dpi=150)
plt.show()

## 3. ΛCDM Limit Validation

**Critical test**: When c_*² = 0, the growth should be exactly scale-independent (standard ΛCDM).

In [ ]:
print("Testing ΛCDM limit (c_*² = 0)...")
passed = validate_lcdm_limit(verbose=True)
print(f"\nValidation {'PASSED ✓' if passed else 'FAILED ✗'}")

## 4. Growth Factor D(k, z) with Msoup Suppression

The modified growth equation:
$$\ddot{\delta}_k + 2H\dot{\delta}_k = 4\pi G \rho_m \delta_k - c_{eff}^2(z) k^2 \delta_k$$

The last term causes scale-dependent suppression at high k.

In [ ]:
# Solve with Msoup suppression
print("Solving growth equation with Msoup suppression...")

params = MsoupParams(c_star_sq=100, Delta_M=0.5, z_t=2.0, w=0.5)
cosmo = CosmologyParams()

solver = MsoupGrowthSolver(
    params, cosmo,
    k_min=0.01, k_max=30, n_k=60,
    z_max=10, n_z=100
)
solution = solver.solve(verbose=True)

print("\nDone!")

In [ ]:
# Plot suppression as function of k at different redshifts
k_grid = solution.k_grid
z_values = [0, 0.5, 1, 2, 5]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Transfer function ratio
for z in z_values:
    T_ratio = solution.transfer_ratio(k_grid, z)
    ax1.semilogx(k_grid, T_ratio, label=f'z = {z}')

ax1.axhline(1, color='k', linestyle='--', alpha=0.3)
ax1.axhline(0.5, color='r', linestyle=':', alpha=0.3, label='Half-mode')
ax1.set_xlabel('k [h/Mpc]')
ax1.set_ylabel(r'$T(k)/T_{\Lambda CDM}(k)$')
ax1.set_title('Transfer Function Suppression')
ax1.legend()
ax1.set_ylim(0, 1.1)

# Power spectrum ratio
for z in z_values:
    P_ratio = solution.power_ratio(k_grid, z)
    ax2.semilogx(k_grid, P_ratio, label=f'z = {z}')

ax2.axhline(1, color='k', linestyle='--', alpha=0.3)
ax2.axhline(0.25, color='r', linestyle=':', alpha=0.3, label='Half-mode (P)')
ax2.set_xlabel('k [h/Mpc]')
ax2.set_ylabel(r'$P(k)/P_{\Lambda CDM}(k)$')
ax2.set_title('Power Spectrum Suppression')
ax2.legend()
ax2.set_ylim(0, 1.1)

plt.tight_layout()
plt.savefig('../results/power_spectrum_suppression.png', dpi=150)
plt.show()

## 5. Suppression as Function of c_*²

How does the suppression scale with the effective sound speed parameter?

In [ ]:
c_star_sq_values = [25, 100, 400, 1600]

fig, ax = plt.subplots(figsize=(10, 6))

for c_sq in c_star_sq_values:
    params_test = MsoupParams(c_star_sq=c_sq, Delta_M=0.5, z_t=2.0, w=0.5)
    solver_test = MsoupGrowthSolver(
        params_test, cosmo,
        k_min=0.01, k_max=30, n_k=50,
        z_max=10, n_z=50
    )
    sol_test = solver_test.solve(verbose=False)
    
    T_ratio = sol_test.transfer_ratio(sol_test.k_grid, z=0)
    ax.semilogx(sol_test.k_grid, T_ratio, 
                label=f'$c_*^2$ = {c_sq} (km/s)$^2$')

ax.axhline(1, color='k', linestyle='--', alpha=0.3)
ax.axhline(0.5, color='r', linestyle=':', alpha=0.5, label='Half-mode')
ax.set_xlabel('k [h/Mpc]')
ax.set_ylabel(r'$T(k)/T_{\Lambda CDM}(k)$ at z=0')
ax.set_title(r'Suppression vs $c_*^2$ Parameter')
ax.legend()
ax.set_ylim(0, 1.1)

plt.tight_layout()
plt.savefig('../results/suppression_vs_c_star.png', dpi=150)
plt.show()

## 6. Halo Mass Function Suppression

The suppressed power spectrum leads to fewer small halos.

In [ ]:
# Compute HMF with Msoup suppression
hmf = HaloMassFunction(cosmo=cosmo, growth_solution=solution)

# Compute at z=0
result = hmf.compute_hmf(z=0, M_min=1e6, M_max=1e15, n_M=80)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Absolute HMF
ax1.loglog(result.M, result.dndlnM, label='Msoup')
ax1.loglog(result.M, result.dndlnM_cdm, '--', label='CDM')
ax1.set_xlabel(r'$M$ [$M_\odot/h$]')
ax1.set_ylabel(r'$dn/d\ln M$ [(h/Mpc)$^3$]')
ax1.set_title('Halo Mass Function at z=0')
ax1.legend()

# Ratio
ax2.semilogx(result.M, result.ratio)
ax2.axhline(1, color='k', linestyle='--', alpha=0.3)
ax2.axhline(0.5, color='r', linestyle=':', alpha=0.5, label='Half suppression')
ax2.set_xlabel(r'$M$ [$M_\odot/h$]')
ax2.set_ylabel(r'HMF$_{Msoup}$ / HMF$_{CDM}$')
ax2.set_title('HMF Suppression Ratio')
ax2.legend()
ax2.set_ylim(0, 1.2)

plt.tight_layout()
plt.savefig('../results/hmf_suppression.png', dpi=150)
plt.show()

In [ ]:
# Half-mode mass
M_hm = hmf.half_mode_mass(z=0)
print(f"Half-mode mass at z=0: M_hm = {M_hm:.2e} M_sun/h")
print(f"                       log10(M_hm) = {np.log10(M_hm):.2f}")

## 7. Redshift Evolution of HMF Suppression

In [ ]:
z_values = [0, 0.5, 1, 2]

fig, ax = plt.subplots(figsize=(10, 6))

for z in z_values:
    result = hmf.compute_hmf(z=z, M_min=1e6, M_max=1e14, n_M=60)
    ax.semilogx(result.M, result.ratio, label=f'z = {z}')

ax.axhline(1, color='k', linestyle='--', alpha=0.3)
ax.axhline(0.5, color='r', linestyle=':', alpha=0.5)
ax.set_xlabel(r'$M$ [$M_\odot/h$]')
ax.set_ylabel(r'HMF$_{Msoup}$ / HMF$_{CDM}$')
ax.set_title('HMF Suppression vs Redshift')
ax.legend()
ax.set_ylim(0, 1.2)

plt.tight_layout()
plt.savefig('../results/hmf_vs_redshift.png', dpi=150)
plt.show()

## 8. Summary

### Validated:
1. ✓ Visibility function V(M; κ) correctly implemented
2. ✓ M_vis(z) turn-on behavior with correct parameters
3. ✓ c_eff²(z) derived from M_vis via sigmoid mapping
4. ✓ Growth equation reduces to ΛCDM when c_*² = 0
5. ✓ Power spectrum suppression at high k
6. ✓ HMF suppression follows from P(k) suppression

### Model Parameters:
- **c_*²**: Controls suppression strength (larger → more suppression at high k)
- **ΔM**: Amplitude of M_vis above threshold
- **z_t**: Transition redshift (when effect turns on)
- **w**: Transition width

### Key Observable:
The half-mode mass M_hm is the scale where the HMF is suppressed by 50%.
This is the primary quantity for comparison with lensing constraints.

In [ ]:
# Save summary
print("="*60)
print("BASELINE VALIDATION COMPLETE")
print("="*60)
print(f"\nModel parameters tested:")
print(f"  c_*² = {params.c_star_sq} (km/s)²")
print(f"  ΔM = {params.Delta_M}")
print(f"  z_t = {params.z_t}")
print(f"  w = {params.w}")
print(f"\nDerived quantities at z=0:")
print(f"  Half-mode mass: M_hm = {M_hm:.2e} M_sun/h")
print(f"  log10(M_hm) = {np.log10(M_hm):.2f}")